In [1]:
import pandas as pd
import numpy as np

In [27]:
df = pd.read_csv("exp_transfer_learning.csv")

In [28]:
df.head(5)

,dataset_name,text,label_name,label_value,label_definition,source,language
0,OLID19,@USER She should ask a few native Americans wh...,subtask_a,OFF,Level A: Offensive language identification \n-...,Twitter,eng
1,OLID19,@USER @USER Go home you’re drunk!!! @USER #MAG...,subtask_a,OFF,Level A: Offensive language identification \n-...,Twitter,eng
2,OLID19,Amazon is investigating Chinese employees who ...,subtask_a,NOT,Level A: Offensive language identification \n-...,Twitter,eng
3,OLID19,"@USER Someone should'veTaken"" this piece of sh...",subtask_a,OFF,Level A: Offensive language identification \n-...,Twitter,eng
4,OLID19,@USER @USER Obama wanted liberals &amp; illega...,subtask_a,NOT,Level A: Offensive language identification \n-...,Twitter,eng


In [36]:
df_target = df[df["dataset_name"] == "EDOS"].copy()
df_target

,dataset_name,text,label_name,label_value,label_definition,source,language
190938,EDOS,"In Nigeria, if you rape a woman, the men rape ...",label_sexist,not sexist,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
190939,EDOS,"Then, she's a keeper. 😉",label_sexist,not sexist,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
190940,EDOS,This is like the Metallica video where the poo...,label_sexist,not sexist,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
190941,EDOS,woman?,label_sexist,not sexist,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
190942,EDOS,I bet she wished she had a gun,label_sexist,not sexist,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
...,...,...,...,...,...,...,...
210933,EDOS,girls really get fucked almost every weekend ?,label_sexist,not sexist,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
210934,EDOS,The hatred for moslems are Reasonable and Just...,label_sexist,not sexist,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
210935,EDOS,Now this is a woman who gets it. 👆,label_sexist,not sexist,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
210936,EDOS,“American Idol” finalist [USER] said nothing i...,label_sexist,not sexist,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng


In [42]:
df_target['label_value'] = df_target.apply(lambda row: 0 if row["label_value"] == "not sexist" else 1, axis=1)
df_target

,dataset_name,text,label_name,label_value,label_definition,source,language
190938,EDOS,"In Nigeria, if you rape a woman, the men rape ...",label_sexist,0,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
190939,EDOS,"Then, she's a keeper. 😉",label_sexist,0,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
190940,EDOS,This is like the Metallica video where the poo...,label_sexist,0,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
190941,EDOS,woman?,label_sexist,0,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
190942,EDOS,I bet she wished she had a gun,label_sexist,0,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
...,...,...,...,...,...,...,...
210933,EDOS,girls really get fucked almost every weekend ?,label_sexist,0,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
210934,EDOS,The hatred for moslems are Reasonable and Just...,label_sexist,0,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
210935,EDOS,Now this is a woman who gets it. 👆,label_sexist,0,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng
210936,EDOS,“American Idol” finalist [USER] said nothing i...,label_sexist,0,TASK A - Binary Sexism Detection: a two-class ...,"Gab, @Reddit",eng


In [37]:
df_aux = df[df["dataset_name"] != "EDOS"].copy()
df_aux

,dataset_name,text,label_name,label_value,label_definition,source,language
0,OLID19,@USER She should ask a few native Americans wh...,subtask_a,OFF,Level A: Offensive language identification \n-...,Twitter,eng
1,OLID19,@USER @USER Go home you’re drunk!!! @USER #MAG...,subtask_a,OFF,Level A: Offensive language identification \n-...,Twitter,eng
2,OLID19,Amazon is investigating Chinese employees who ...,subtask_a,NOT,Level A: Offensive language identification \n-...,Twitter,eng
3,OLID19,"@USER Someone should'veTaken"" this piece of sh...",subtask_a,OFF,Level A: Offensive language identification \n-...,Twitter,eng
4,OLID19,@USER @USER Obama wanted liberals &amp; illega...,subtask_a,NOT,Level A: Offensive language identification \n-...,Twitter,eng
...,...,...,...,...,...,...,...
190933,Ethos,From the midnight sun where the hot springs blow,isHate,0.0,NaN,Multi,eng
190934,Ethos,Don't say I'm not your type,isHate,0.0,NaN,Multi,eng
190935,Ethos,And therefore never send to know for whom the...,isHate,0.0,NaN,Multi,eng
190936,Ethos,And I can't stand another day,isHate,0.0,NaN,Multi,eng


In [38]:
unique_values = df_aux.groupby("dataset_name")['label_value'].apply(lambda x: list(x.unique()))
unique_values

dataset_name
DGHD                                        [hate, nothate]
Ethos     [1.0, 0.983870967741936, 0.983606557377049, 0....
MHS       [-3.9, -6.52, 0.36, 0.26, 1.54, -4.93, 0.17, 2...
OLID19                                           [OFF, NOT]
Name: label_value, dtype: object

In [39]:
def map_label_value(row):
    if row["dataset_name"] == "DGHD":
        if row["label_value"] == "hate":
            return 1
    
    elif row["dataset_name"] == "OLID":
        if row["label_value"] == "OFF":
            return 1
    
    elif row["dataset_name"] == "MHS":
        if float(row["label_value"]) > 0.5:
            return 1
    
    elif row["dataset_name"] == "Ethos":
        if float(row["label_value"]) >= 0.5:
            return 1
    
    return 0

In [40]:
df_aux["label_value"] = df.apply(map_label_value,axis=1)

In [41]:
df_aux

,dataset_name,text,label_name,label_value,label_definition,source,language
0,OLID19,@USER She should ask a few native Americans wh...,subtask_a,0,Level A: Offensive language identification \n-...,Twitter,eng
1,OLID19,@USER @USER Go home you’re drunk!!! @USER #MAG...,subtask_a,0,Level A: Offensive language identification \n-...,Twitter,eng
2,OLID19,Amazon is investigating Chinese employees who ...,subtask_a,0,Level A: Offensive language identification \n-...,Twitter,eng
3,OLID19,"@USER Someone should'veTaken"" this piece of sh...",subtask_a,0,Level A: Offensive language identification \n-...,Twitter,eng
4,OLID19,@USER @USER Obama wanted liberals &amp; illega...,subtask_a,0,Level A: Offensive language identification \n-...,Twitter,eng
...,...,...,...,...,...,...,...
190933,Ethos,From the midnight sun where the hot springs blow,isHate,0,NaN,Multi,eng
190934,Ethos,Don't say I'm not your type,isHate,0,NaN,Multi,eng
190935,Ethos,And therefore never send to know for whom the...,isHate,0,NaN,Multi,eng
190936,Ethos,And I can't stand another day,isHate,0,NaN,Multi,eng


In [43]:
len(df_aux[df_aux["label_value"]==1]) / len(df_aux)

0.3752841236422294

In [44]:
len(df_target[df_target["label_value"]==1]) / len(df_target)

0.2427

In [50]:
from Sampler import balanced_fixedcount

aux_train = balanced_fixedcount(df_aux, 100000, "label_value", "dataset_name")
aux_train

,dataset_name,text,label_name,label_value,label_definition,source,language
0,DGHD,Young Womens Trust needs to be destroyed or t...,label,0,"a binary variable, indicating whether or not t...",NaN,eng
1,MHS,@DoingRight1 Why did anyone think allowing hor...,hate_speech_score,1,"Continuous hate speech measure, where higher =...","YouTube, Twitter, and Reddit",eng
2,MHS,Life without Allah is meaningless.,hate_speech_score,0,"Continuous hate speech measure, where higher =...","YouTube, Twitter, and Reddit",eng
3,DGHD,this is a Fucken blaady madlad,label,0,"a binary variable, indicating whether or not t...",NaN,eng
4,MHS,"Okay, r/niceguys front man. Who's the incel ag...",hate_speech_score,0,"Continuous hate speech measure, where higher =...","YouTube, Twitter, and Reddit",eng
...,...,...,...,...,...,...,...
99995,MHS,@libertyladyusa Deport this trash,hate_speech_score,1,"Continuous hate speech measure, where higher =...","YouTube, Twitter, and Reddit",eng
99996,MHS,Ha! Take that nigger,hate_speech_score,0,"Continuous hate speech measure, where higher =...","YouTube, Twitter, and Reddit",eng
99997,MHS,Hitler didn't finish it. Can u. If a nigger ur...,hate_speech_score,1,"Continuous hate speech measure, where higher =...","YouTube, Twitter, and Reddit",eng
99998,Ethos,When lift your arms do your ‚Äúboobs‚Äù go awa...,isHate,0,NaN,Multi,eng


In [51]:
len(aux_train[aux_train["label_value"]==1]) / len(aux_train)

0.45541

## Target Split

In [54]:
from sklearn.model_selection import train_test_split
train_tdf, test_tdf = train_test_split(df_target, test_size=0.2, stratify=df_target['label_value'], random_state=42)

In [57]:
from Sampler import balanced_downsampling
print("Before Downsampling: ",len(test_tdf[test_tdf["label_value"]==1]) / len(test_tdf))

resampled_test_tdf = balanced_downsampling(test_tdf, "label_value")

print("Before Downsampling: ",len(resampled_test_tdf[resampled_test_tdf["label_value"]==1]) / len(resampled_test_tdf))

Before Downsampling:  0.24275
Before Downsampling:  0.5


In [ ]:
for train_per in [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    _, train_tdf_slice = train_test_split(train_tdf, test_size = train_per, stratify=train_tdf['label_value'], random_state=42)